# Running the prompts against several models for code quality comparison

Models:
- GPT 4.1
- GPT 4o
- GPT o3
- GPT o1
- Claude Sonnet 3.7
- Claude Sonnet 4.0
- Deepseek

Keeping default temperature to simulate consumer usage in chat apps like ChatGPT or Anthropics interface
Adding "Only output one alternative to solve this." in order to prevent the model to output multiple suggestions which are harder to evaluate.

Annotating the prompts by characteristics:
- Personification score
- request type (question vs command)
- length?
-

In [1]:
import sqlite3
import pandas as pd
import os

db_path = "../../giicg.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file does not exist: {db_path}")

conn = sqlite3.connect(db_path)
prompts = pd.read_sql("SELECT * FROM translated_scratch_prompts", conn)
prompts = prompts.drop(columns=['conversational', 'code', 'other'])
prompts

,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,python,en
7,18,266,user,how can I combine two grib files in jupyter no...,Woman (cisgender),60,python,de
8,20,268,user,I work with Python and have to read an NMEA fi...,Woman (cisgender),65,python,de
9,21,290,user,please write method to unzip file in python,Woman (cisgender),73,python,en


## Open AI & Deepseek

In [2]:
from openai import OpenAI

gpt_client = OpenAI()
#"o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "gpt-4o-2024-08-06"
gpt_models = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14"]

deepseek_client = OpenAI(api_key=os.environ.get("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")
deepseek_models = ["deepseek-chat"]

def prompt_gpt(client, model, prompt):
    response = client.responses.create(
        model=model,
        input=prompt,
    )
    return response.output_text

for model in gpt_models:
    prompts[model] = prompts["message_text"].apply(lambda x: prompt_gpt(gpt_client, model, x))

def prompt_deepseek(client, model, prompt):
    response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": f"{prompt}. Only output one alternative to solve this.",}
            ],
            #max_tokens=1000,
            #temperature=0.0
        )

    return response.choices[0].message.content


# for model in deepseek_models:
#     prompts[model] = prompts["message_text"].apply(lambda x: prompt_deepseek(deepseek_client, model, x))

prompts



,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language,chatgpt-4o-latest,o3-2025-04-16,gpt-5-chat-latest,gpt-4.1-2025-04-14
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en,To use **Dummy Hot Encoding** on the `'embarke...,In scikit-learn you do one-hot (a.k.a. dummy-h...,"Great question! \nIn scikit-learn, the equiva...",To perform Dummy Hot Encoding (commonly known ...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en,To encode and compress a JavaScript object int...,When somebody says “encode a JavaScript object...,Great question! If you want to take a JavaScri...,To **encode and compress** a JavaScript object...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en,Certainly! You can extract the `persona` colum...,Sure – the idea is to turn every “key: value” ...,Yes! You can split this structured string into...,Absolutely! You can split the `persona` column...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en,That’s a great idea — and quite a common pain ...,"Your idea is practical, very doable, and—if yo...",That’s a great idea — it’s a common pain point...,Your idea is excellent and quite practical—man...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en,Certainly! You can use the Blender API (`bpy`)...,Below is a small stand-alone Python script tha...,"Got it! You want to ""reconstruct"" a collection...",Certainly! If you have a collection of complex...
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en,To **run a Python `Future` or `coroutine` with...,With asyncio you don’t have to “part-run’’ a F...,You're describing a usage pattern that looks l...,Great question! You want to **run a Python cor...
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,python,en,Sure! Here's a short Python script you can use...,Here’s a compact cell you can drop straight in...,"Got it! You want to load `data.csv`, and for t...",Absolutely! Here’s a short Python script for y...
7,18,266,user,how can I combine two grib files in jupyter no...,Woman (cisgender),60,python,de,To **combine two GRIB files** in a Jupyter Not...,Below is the shortest-to-longest set of soluti...,Great question! Combining GRIB files programma...,You **can combine two GRIB files** in a Jupyte...
8,20,268,user,I work with Python and have to read an NMEA fi...,Woman (cisgender),65,python,de,To help you work with the NMEA file `Task3_NME...,Below is a small “cook-book” that should cover...,Got it ✅ \n\nYou’ve got a file called **`Task...,Great! Working with NMEA files (which are esse...
9,21,290,user,please write method to unzip file in python,Woman (cisgender),73,python,en,Sure! Here's a simple Python method to **unzip...,"Below is a compact, reusable helper that extra...","Sure! In Python, you can unzip files using the...",Certainly! Here’s a simple method to **unzip**...


## Anthropic

In [3]:
from anthropic import Anthropic
import dotenv
dotenv.load_dotenv()

models = [ "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]
claude_client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

def prompt_claude(client, model, prompt):
    response = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": f"{prompt}. Only output one alternative to solve this.",
        }
    ],
    model=model,
    #temperature=0.0  #Note that even with temperature of 0.0, the results will not be fully deterministic.
    )
    return "".join(block.text for block in response.content if hasattr(block, "text"))

for model in models:
    prompts[model] = prompts["message_text"].apply(lambda x: prompt_claude(claude_client, model, x))

In [4]:
prompts

,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language,chatgpt-4o-latest,o3-2025-04-16,gpt-5-chat-latest,gpt-4.1-2025-04-14,claude-3-7-sonnet-20250219,claude-sonnet-4-20250514
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en,To use **Dummy Hot Encoding** on the `'embarke...,In scikit-learn you do one-hot (a.k.a. dummy-h...,"Great question! \nIn scikit-learn, the equiva...",To perform Dummy Hot Encoding (commonly known ...,# Using One-Hot Encoding for the 'embarked' co...,You can use `OneHotEncoder` from sklearn:\n\n`...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en,To encode and compress a JavaScript object int...,When somebody says “encode a JavaScript object...,Great question! If you want to take a JavaScri...,To **encode and compress** a JavaScript object...,# URL-Safe Encoding and Compression for JavaSc...,Use `btoa()` and `atob()` with JSON serializat...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en,Certainly! You can extract the `persona` colum...,Sure – the idea is to turn every “key: value” ...,Yes! You can split this structured string into...,Absolutely! You can split the `persona` column...,Here's a solution to separate the persona colu...,You can use this approach to split the persona...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en,That’s a great idea — and quite a common pain ...,"Your idea is practical, very doable, and—if yo...",That’s a great idea — it’s a common pain point...,Your idea is excellent and quite practical—man...,# PDF Auto-Renaming Tool Evaluation\n\nYour id...,Your idea is excellent and very practical for ...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en,Certainly! You can use the Blender API (`bpy`)...,Below is a small stand-alone Python script tha...,"Got it! You want to ""reconstruct"" a collection...",Certainly! If you have a collection of complex...,# Blender Script: Replicating Meshes with Cube...,Here's a Python script for Blender that create...
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en,To **run a Python `Future` or `coroutine` with...,With asyncio you don’t have to “part-run’’ a F...,You're describing a usage pattern that looks l...,Great question! You want to **run a Python cor...,# Running a Python Future Without Blocking\n\n...,You can use `asyncio.create_task()` to run a c...
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,python,en,Sure! Here's a short Python script you can use...,Here’s a compact cell you can drop straight in...,"Got it! You want to load `data.csv`, and for t...",Absolutely! Here’s a short Python script for y...,```python\nimport pandas as pd\nimport nltk\nf...,Here's a Python script for your Jupyter notebo...
7,18,266,user,how can I combine two grib files in jupyter no...,Woman (cisgender),60,python,de,To **combine two GRIB files** in a Jupyter Not...,Below is the shortest-to-longest set of soluti...,Great question! Combining GRIB files programma...,You **can combine two GRIB files** in a Jupyte...,# Combining Two GRIB Files in Jupyter Notebook...,You can use the `xarray` library with the `cfg...
8,20,268,user,I work with Python and have to read an NMEA fi...,Woman (cisgender),65,python,de,To help you work with the NMEA file `Task3_NME...,Below is a small “cook-book” that should cover...,Got it ✅ \n\nYou’ve got a file called **`Task...,Great! Working with NMEA files (which are esse...,# Reading an NMEA File in Python\n\nHere's a s...,Here's one way to read an NMEA file in Python:...
9,21,290,user,please write method to unzip file in python,Woman (cisgender),73,python,en,Sure! Here's a simple Python method to **unzip...,"Below is a compact, reusable helper that extra...","Sure! In Python, you can unzip files using the.

In [45]:
import re

def parse_code_snippets(text):
    """
    Extract code snippets marked with triple backticks and any of these language tags:
    python, javascript, js, typescript, ts.
    Joins all found code into a single string separated by newlines.
    """
    pattern = r"```(?:python|javascript|js|typescript|ts|html)[ \t]*\n(.*?)(?=```)"
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    code_snippets = [m.strip() for m in matches]
    return "\n".join(code_snippets)


columns = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]

for column in columns:
    prompts[f"code_{column}"] = prompts[column].apply(parse_code_snippets)

prompts

,conversation_id,message_id,role,message_text,gender,user_id,programming_language,language,chatgpt-4o-latest,o3-2025-04-16,gpt-5-chat-latest,gpt-4.1-2025-04-14,claude-3-7-sonnet-20250219,claude-sonnet-4-20250514,code_chatgpt-4o-latest,code_o3-2025-04-16,code_gpt-5-chat-latest,code_gpt-4.1-2025-04-14,code_claude-3-7-sonnet-20250219,code_claude-sonnet-4-20250514
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,python,en,To perform **Dummy (One-Hot) Encoding** on the...,With scikit-learn you can turn the string valu...,Great question! If you want to apply **dummy (...,Certainly! To perform **dummy hot encoding** (...,# Dummy Hot Encoding for 'embarked' Column wit...,You can use sklearn's `OneHotEncoder` to creat...,import pandas as pd\n\ndata = pd.DataFrame({\n...,import pandas as pd\nfrom sklearn.preprocessin...,import pandas as pd\nfrom sklearn.preprocessin...,import pandas as pd\n\ndf = pd.DataFrame({\n ...,import pandas as pd\nimport numpy as np\nfrom ...,from sklearn.preprocessing import OneHotEncode...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,javascript,en,To encode and compress a JavaScript object int...,Below is a pragmatic “toolbox” you can copy-an...,When you want to serialize a JavaScript object...,To **encode and compress a JavaScript object i...,# Encoding and Compressing JavaScript Objects ...,Here are the best approaches for encoding and ...,const jsonString = JSON.stringify(obj);\nimpor...,,"const obj = { foo: ""bar"", count: 42 };\n\n// C...","import { compressToEncodedURIComponent, decomp...",// Encoding\nfunction encodeObject(obj) {\n c...,function encodeObject(obj) {\n return btoa(JS...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,python,en,To separate the `persona` column in your panda...,Yes. \nBecause every attribute inside persona...,Got it ✅ — you basically want to take that `pe...,Absolutely! You want to parse the `persona` co...,I can help you extract the persona information...,You can extract and separate the `persona` col...,import pandas as pd\n\n# Assume your original ...,import pandas as pd\n\n# --- example data ----...,import pandas as pd\n\n# Example DataFrame\nda...,import pandas as pd\n\n# Example: If you alrea...,import pandas as pd\nimport re\n\n# Assuming d...,import pandas as pd\n\ndef parse_persona(perso...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,python,en,Your idea is **excellent** and very practical ...,Below is a structured “technical review” of yo...,That’s a really good and practical idea 🚀 — ma...,Your idea is not only feasible but also a high...,# Evaluating Your PDF Auto-Renaming Project Id...,Your idea is excellent and very practical! As ...,import arxiv\n\nsearch = arxiv.Search(id_list=...,#!/usr/bin/env python3\n# rename_papers.py\n\n...,import os\nimport re\nimport fitz # PyMuPDF\n...,import os\nfrom PyPDF2 import PdfReader\n\ndef...,import os\nimport re\nfrom PyPDF2 import PdfRe...,
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,python,en,Sure! Below is a **Blender Python (bpy)** scri...,The script below scans one of your collections...,Got it. You want to take a collection of objec...,Let's break down the task and provide a **Blen...,# Blender Script: Replace Meshes with Cube Ins...,Here's a Blender Python script that will creat...,import bpy\nimport mathutils\n\n# === CONFIGUR...,import bpy\nimport mathutils\nimport bmesh\n\n...,import bpy\nimport mathutils\n\n# Parameters\n...,import bpy\nfrom mathutils import Matrix\n\n# ...,import bpy\nimport mathutils\nfrom math import...,
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,python,en,To run a Python `Future` or `coroutine` non-bl...,With `asyncio` a coroutine is driven by the ev...,What you're describing is essentially: \n\n- K...,You're asking about **progressive execution of...,# Running Python Futures Wi

## Pivot columns for saving

In [6]:

model_columns = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]

melted = prompts.melt(
    id_vars=["conversation_id", "message_id", "message_text", "gender", "user_id", "programming_language"],
    value_vars=model_columns,
    var_name="model_version",
    value_name="model_reply"
)

# code_columns = [f"code_{col}" for col in model_columns if f"code_{col}" in prompts.columns]
#
# if code_columns:
#     code_melted = prompts.melt(
#         id_vars=["conversation_id", "message_id", "message_text", "gender", "user_id", "programming_language"],
#         value_vars=code_columns,
#         var_name="model_version_code",
#         value_name="code"
#     )
#
#     code_melted["model_version"] = code_melted["model_version_code"].str.replace("code_", "")
#
#     melted = melted.merge(
#         code_melted[["conversation_id", "message_id", "model_version", "code"]],
#         on=["conversation_id", "message_id", "model_version"],
#         how="left"
#     )
# else:
#
#     melted["code"] = None


melted = melted.reset_index(drop=True)
melted["code_reply_id"] = melted.index + 1


final_df = melted[
    ["code_reply_id", "conversation_id", "message_id", "message_text", "gender", "user_id",
     "model_reply", "model_version", "programming_language"]
]


final_df

,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,model_version,programming_language
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,To use **Dummy Hot Encoding** on the `'embarke...,chatgpt-4o-latest,python
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,To encode and compress a JavaScript object int...,chatgpt-4o-latest,javascript
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,Certainly! You can extract the `persona` colum...,chatgpt-4o-latest,python
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,That’s a great idea — and quite a common pain ...,chatgpt-4o-latest,python
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,Certainly! You can use the Blender API (`bpy`)...,chatgpt-4o-latest,python
...,...,...,...,...,...,...,...,...,...
199,200,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies the `parse_obj` fu...,claude-sonnet-4-20250514,python
200,201,72,1534,can you write me some python code to count the...,Woman (cisgender),11,```python\nimport json\n\ndef count_objects_in...,claude-sonnet-4-20250514,python
201,202,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge the two dataframes using the 'cl...,claude-sonnet-4-20250514,python
202,203,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,Use the **Elbow Method** with Within-Cluster S...,claude-sonnet-4-20250514,python


## Save to database

In [7]:
final_df.to_sql("translated_scratch_prompts_with_code", conn, if_exists="replace", index=False)

204